Эта семинарская тетрадка дополняет слайды презентации, посвященные моделям **мешка слов** и **tf-idf**.  

Для начала построим модель мешка слов на известных уже нам строфах из Пушкина.

In [ ]:
text = 'Мой дядя самых честных правил, Когда не в шутку занемог, Он уважать себя заставил И лучше выдумать не мог. Его пример другим наука; Но, боже мой, какая скука С больным сидеть и день и ночь, Не отходя ни шагу прочь! Какое низкое коварство Полуживого забавлять, Ему подушки поправлять, Печально подносить лекарство, Вздыхать и думать про себя: Когда же чёрт возьмёт тебя?'

Текст, который мы подадим для создания модели текста, лучше предварительно обработать. Например, удалить стоп-слова и лемматизировать.

In [ ]:
from nltk.corpus import stopwords 
import re
from nltk import word_tokenize
stopwords_ru = stopwords.words('russian')

In [ ]:
text = re.sub('[^а-яА-ЯёЁ -]', '', text.lower())
print(text)

Лемматизируем:

In [ ]:
from pymorphy3 import MorphAnalyzer 
morph = MorphAnalyzer() 

In [ ]:
lemmatized_text = [morph.parse(tok)[0].normal_form for tok in word_tokenize(text)]
print(lemmatized_text)

Удалим стоп-слова (да, обычно их лучше удалять после лемматизации):

In [ ]:
text_no_stop = [' '.join([token for token in lemmatized_text if token not in stopwords_ru])]
print(text_no_stop) #мы склеили обратно и поместили в список, потому что векторизатор в sklearn принимает список строк

Модель мешка слов очень легко создается с помощью библиотеки **sklearn**.

In [ ]:
# Создаём модель мешка слов
from sklearn.feature_extraction.text import CountVectorizer #импорт функции, которая создаст вектора мешка слов
vectorizer = CountVectorizer() #сохраним в переменную
X = vectorizer.fit_transform(text_no_stop) #fit transform cоздает модель мешка слов
print(X) #пока тут ничего полезного

Теперь получим список уникальных слов, который входит в мешок слов:

In [ ]:
vectorizer.get_feature_names_out()

Чтобы получить значения в матрице мешка слов, нужно сделать следующее:

In [ ]:
X.toarray()[0]

И сделаем теперь вектор наглядным:

In [ ]:
import pandas as pd

In [ ]:
text_vector = pd.DataFrame({'words': vectorizer.get_feature_names_out(),
                            'vectors': X.toarray()[0]})
text_vector

Здесь единый и короткий текст, где все леммы являются гапаксами. Давайте возьмем корпус чуть больше, при этом это будут разные документы.

In [ ]:
with open('C:\\Users\\Aleksandr\\Downloads\\EugeneOnegin.txt', encoding='utf-8') as txt:
    text = txt.read()
    corpus = re.split(r'ГЛАВА \w+\b', text)
    print(len(corpus))
    clean_texts = []
    for text in corpus:
        text = re.sub(r'\n', ' ', text)
        text = re.sub('[^а-яА-ЯёЁ -]', '', text.lower())
        lemmatized_text = [morph.parse(tok)[0].normal_form for tok in word_tokenize(text)]
        text_no_stop = ' '.join([token for token in lemmatized_text if token not in stopwords_ru])
        clean_texts.append(text_no_stop)
print(clean_texts)

Теперь составим векторные модели мешка слов для каждой из глав:

In [ ]:
X = vectorizer.fit_transform(clean_texts) #fit transform cоздает модель мешка слов
len(vectorizer.get_feature_names_out()) #количество уникальных лемм в нашей модели

In [ ]:
print(list(vectorizer.get_feature_names_out()))

In [ ]:
X.toarray()

Зададим таблицу несколько иначе, потому что вектора текстов удобнее располагать другим образом:

In [ ]:
text_vector = pd.DataFrame(columns = vectorizer.get_feature_names_out(), data = X.toarray()) 
text_vector #такие данные называются sparse data, об этом подробнее вы узнаете в курсе по машинному обучению

Таким образом вы можете извлечь вектор любого интересующего вас текста в коллекции:

In [ ]:
text_vector.loc[5]

И вектор любого интересующего слова:

In [ ]:
print(text_vector['онегин'].tolist())

In [ ]:
print(text_vector['татьяна'].tolist())

**Задание 1**. Сделайте список из двух небольших текстов и получите их вектора.

Посчитаем косинусное расстояние: 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
vector_1 = [X.toarray()[1]]
vector_2 = [X.toarray()[5]]
cosine_similarity(vector_1, vector_2)

У векторных моделей широкое применение, особенно в машинном обучении, например, в задаче классификации текстов. Посмотрим на простой классификатор при помощи мешка слов.  
**NB! От вас не требуется сейчас понимания и умения писать классификаторы, это только иллюстрация того, зачем вообще могут быть нужны векторные модели текста.**

In [ ]:
data = pd.read_csv("C:\\Users\\Aleksandr\\Downloads\\archive\\spam.csv", encoding='latin-1') #добавьте свой путь

In [ ]:
data.head()

Немного предобработакм датасет: удалим лишние столбцы и измененим метки спама.

In [ ]:
#немного предобработки датасета: удаление лишних столбцов и изменение меток спама
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"label", "v2":"text"})
data['label'] = data.label.map({'ham':0, 'spam':1})

In [ ]:
data.head()

In [ ]:
#количество спама и не-спама
data.label.value_counts()

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

Также немного предобработаем наш корпус.

In [ ]:
corpus = []
for i in range(0, len(data)): #проходимся по каждой строке
    review = re.sub('[^a-zA-Z ]', '', data['text'][i].lower()) #удаляем все, что не латиница и не пробел
    review = [ps.stem(word) for word in word_tokenize(review) if not word in set(stopwords.words('english'))] #стемминг
    review = ' '.join(review) #склеиваем: CountVectorizer из sklearn требует строковых данных
    corpus.append(review)

In [ ]:
print(corpus)

In [ ]:
# Создаём модель мешка слов
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray() #полученные вектора
y = data.iloc[:, 0].values #метки спама

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
print(len(X))
#print(len(X[1])) #количество типов

In [ ]:
print(X)
#print(y)

In [ ]:
# Fitting Naive Bayes to the Training set (Gaussian NB)
prediction = dict()
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
classifier = GaussianNB()
classifier.fit(X_train, y_train)

In [ ]:
# Predicting the Test set results
prediction["GaussianNB"] = classifier.predict(X_test)
accuracy_score(y_test,prediction["GaussianNB"])

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

In [ ]:
# Предсказание на мультиномиальном наивном байесовском алгоритме
prediction["MultinomialNB"] = classifier.predict(X_test)
accuracy_score(y_test,prediction["MultinomialNB"])

In [ ]:
print(classification_report(y_test, prediction['MultinomialNB'], target_names = ["Ham", "Spam"]))

<b>Поработаем с tf-idf.</b> Разберем для начала игрушечный пример.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
text_1 = 'книга ужасно интересная'
text_2 = 'фильм ужасно увлекательный'
texts = [text_1, text_2]

Получим вектора токенов в этих текстах:

In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1)) #ngram_range=(1, 1) означает униграммы, (1, 2) - униграммы и биграммы, (2, 2) - биграммы и т.д.

In [ ]:
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
tfidf_matrix

In [ ]:
npm_tfidf = tfidf_matrix.todense()
document_1_vector = npm_tfidf[0] #вектор первого текста
document_2_vector = npm_tfidf[1] #вектор второго текста
print(tfidf_vectorizer.get_feature_names_out()) #токены вектора
print(document_1_vector)
print(document_2_vector)

In [ ]:
x_1 = document_1_vector.tolist()
x_2 = document_2_vector.tolist()

In [ ]:
df = pd.DataFrame(data=[x_1[0], x_2[0]], columns=tfidf_vectorizer.get_feature_names_out())

In [ ]:
df

Вернемся к Пушкину.

In [ ]:
with open('C:\\Users\\Aleksandr\\Downloads\\EugeneOnegin.txt', encoding='utf-8') as txt:
    text = txt.read()
    corpus = re.split(r'ГЛАВА \w+\b', text)
    print(len(corpus))
    clean_texts = []
    for text in corpus:
        text = re.sub(r'\n', ' ', text)
        text = re.sub('[^а-яА-ЯёЁ -]', '', text.lower())
        lemmatized_text = [morph.parse(tok)[0].normal_form for tok in word_tokenize(text)]
        text_no_stop = ' '.join([token for token in lemmatized_text if token not in stopwords_ru])
        clean_texts.append(text_no_stop)
print(clean_texts)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X = tfidf_vectorizer.fit_transform(clean_texts) #полученные tf-idf вектора 

In [ ]:
import pandas as pd

In [ ]:
text_vector = pd.DataFrame(columns = tfidf_vectorizer.get_feature_names_out(), data = X.toarray()) 
text_vector 

И так же, как и с мешком слов, можем получать вектора отдельных слов и текстов:

In [ ]:
print(text_vector['сплин'])

Извлечем ключевые слова для первой главы. Для начала объединим леммы со значениями tf-idf для первой главы.

In [ ]:
lemmas = list(text_vector.columns)
tf_idf = text_vector.loc[0].tolist()
lemmas_tf_idf = list(zip(lemmas, tf_idf))
lemmas_tf_idf

Осталось только отсортировать:

In [ ]:
sorted_chapter_one = sorted(lemmas_tf_idf, key=lambda x: x[1], reverse = 'True')
sorted_chapter_one[:20]

Посмотрим на последнюю главу:

In [ ]:
lemmas = list(text_vector.columns)
tf_idf = text_vector.loc[7].tolist()
lemmas_tf_idf = list(zip(lemmas, tf_idf))
sorted_chapter_eight = sorted(lemmas_tf_idf, key=lambda x: x[1], reverse = 'True')
sorted_chapter_eight[:20]

Результат, конечно, далек от совершенства. Отчасти потому, что это довольно искусственное разбиение на отдельные тексты (главы), отчасти потому, что мы могли бы добавить больше слоев предобработки. Например, оставив только существительные. Попробуем:

In [ ]:
with open('C:\\Users\\Aleksandr\\Downloads\\EugeneOnegin.txt', encoding='utf-8') as txt:
    text = txt.read()
    corpus = re.split(r'ГЛАВА \w+\b', text)
    print(len(corpus))
    clean_texts = []
    for text in corpus:
        text = re.sub(r'\n', ' ', text)
        text = re.sub('[^а-яА-ЯёЁ -]', '', text.lower())
        #чуть-чуть изменим предобработку, добавив одно условие:
        lemmatized_text = [morph.parse(tok)[0].normal_form for tok in word_tokenize(text) if morph.parse(tok)[0].tag.POS == 'ADJF']
        text_no_stop = ' '.join([token for token in lemmatized_text if token not in stopwords_ru])
        clean_texts.append(text_no_stop)
print(clean_texts)

In [ ]:
X = tfidf_vectorizer.fit_transform(clean_texts) #полученные tf-idf вектора 
text_vector = pd.DataFrame(columns = tfidf_vectorizer.get_feature_names_out(), data = X.toarray()) 

In [ ]:
lemmas = list(text_vector.columns)
tf_idf_one = text_vector.loc[0].tolist()
tf_idf_eight = text_vector.loc[7].tolist()
lemmas_tf_idf_one = list(zip(lemmas, tf_idf_one))
lemmas_tf_idf_eight = list(zip(lemmas, tf_idf_eight))
sorted_chapter_one = sorted(lemmas_tf_idf_one, key=lambda x: x[1], reverse = True)
sorted_chapter_eight = sorted(lemmas_tf_idf_eight, key=lambda x: x[1], reverse = True)

In [ ]:
sorted_chapter_one[:20]

In [ ]:
sorted_chapter_eight[:20]

**Задание 1**. Сравните ключевые слова во второй главе (дружба Онегина и Ленского) и в шестой главе (убийство Ленского Онегиным).

In [ ]:
lemmas = list(text_vector.columns)
tf_idf_two = text_vector.loc[1].tolist()
tf_idf_six = text_vector.loc[5].tolist()
lemmas_tf_idf_two = list(zip(lemmas, tf_idf_two))
lemmas_tf_idf_six = list(zip(lemmas, tf_idf_six))
sorted_chapter_two = sorted(lemmas_tf_idf_two, key=lambda x: x[1], reverse = True)
sorted_chapter_six = sorted(lemmas_tf_idf_six, key=lambda x: x[1], reverse = True)

In [ ]:
sorted_chapter_two[:20]

In [ ]:
sorted_chapter_six[:20]

**Задание 2**. Соберите свой небольшой корпус из 3-4 текстов (например, 3-4 разные статьи из википедии на различные темы). Постройте tf-idf модель и сравните ключевые слова.

**Задание 3 (со звездочкой)**. Модифицируйте код для предыдущего задания так, чтобы в ключевых словах были только глаголы. 